In [1]:
# -*- coding: utf-8 -*-
#setup 
import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace import sarimax
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA 
import math

os.chdir("C:/Users/arora/Documents/spark-python")  
os.environ['SPARK_HOME'] = 'C:/Users/arora/spark-2.3.3-bin-hadoop2.7'

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. 
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.7-src.zip"))

from pyspark.sql import Row
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import split
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql.functions import lit
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import TimestampType,IntegerType
from pyspark.sql.functions import lag
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from datetime import datetime
from pyspark.ml.clustering import KMeans

def time_delta(y,x): 
    from datetime import datetime
    end = datetime.strptime(y, '%Y-%m-%dT%H:%M:%S.%fZ')  
    start = datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ')
    delta = (end-start).total_seconds()
    return delta

In [10]:
pathToTextFile = r'''C:\Users\arora\WeblogChallenge\data\2015_07_22_mktplace_shop_web_log_sample.log.gz'''  
df = pd.read_csv(pathToTextFile, sep=" ", names=["timestamp_str","elb_name","client_ip","back_end_ip","request_processing_time","backend_processing_time",\
          "response_processing_time","elb_status_code","backend_status_code","received_bytes","sent_bytes",\
          "request","user_agent","ssl_cipher","ssl_protocol"])
df['user_agent'] = df['user_agent'].astype(str)
df['url'] = df['request'].astype(str).str.split().str[1]
df['IP']=df['client_ip'].astype(str).str.split(':').str[0]
df.to_csv(r'''C:\Users\arora\WeblogChallenge\data\2015_07_22_mktplace_shop_web_log_sample.log.csv''')
# DataFramePrediction = SpSession.createDataFrame(df)

In [11]:
#Create a Spark Session
SpSession = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Nidhi") \
    .getOrCreate()
    
#Get the Spark Context from Spark Session    
sc = SpSession.sparkContext


In [18]:
#add data to dataframe
logDataFrame = SpSession.read.csv(r'''C:\Users\arora\WeblogChallenge\data\2015_07_22_mktplace_shop_web_log_sample.log.csv'''  ,header=True,sep=",");
logDataFrame.show(2)

+---+--------------------+----------------+--------------------+-------------+-----------------------+-----------------------+------------------------+---------------+-------------------+--------------+----------+--------------------+--------------------+--------------------+------------+--------------------+---------------+
|_c0|       timestamp_str|        elb_name|           client_ip|  back_end_ip|request_processing_time|backend_processing_time|response_processing_time|elb_status_code|backend_status_code|received_bytes|sent_bytes|             request|          user_agent|          ssl_cipher|ssl_protocol|                 url|             IP|
+---+--------------------+----------------+--------------------+-------------+-----------------------+-----------------------+------------------------+---------------+-------------------+--------------+----------+--------------------+--------------------+--------------------+------------+--------------------+---------------+
|  0|2015-07-22T09:

In [24]:
#For creating sessions a column is created for lagged timestamps partitioned over IP, user_agent
func =  udf (lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'), TimestampType())

func2 =  udf (lambda x: x.replace(microsecond=0), TimestampType())

logDataFrame.createOrReplaceTempView("data_With_IP")
logDataFrameWithIP = logDataFrame.withColumn('time_stamp', func(col('timestamp_str'))).cache()

logDataFrameWithIPNoMicroSecs = logDataFrameWithIP.withColumn('time_stamp_without_microsecs', func2(col('time_stamp'))).cache()
logDataFrameWithIPNoMicroSecs.take(1)
logDataFrameWithIPNoMicroSecs.createOrReplaceTempView("df_time_stamp_without_microsecs")

logDataFrameIPTimeStamp = logDataFrameWithIP.withColumn('time_stamp_previous',
                        lag(logDataFrameWithIP.time_stamp,1)
                                 .over(Window.partitionBy("IP","user_agent").orderBy("time_stamp"))).cache()

In [28]:
logDataFrameIPTimeStamp

DataFrame[_c0: string, timestamp_str: string, elb_name: string, client_ip: string, back_end_ip: string, request_processing_time: string, backend_processing_time: string, response_processing_time: string, elb_status_code: string, backend_status_code: string, received_bytes: string, sent_bytes: string, request: string, user_agent: string, ssl_cipher: string, ssl_protocol: string, url: string, IP: string, time_stamp: timestamp, time_stamp_previous: timestamp, time_diff_in_secs: bigint]

In [25]:
#difference in current and previous timestamp gives time spent in seconds
logDataFrameIPTimeStamp = logDataFrameIPTimeStamp.withColumn(
    "time_diff_in_secs", 
    unix_timestamp("time_stamp") - unix_timestamp("time_stamp_previous")
)

logDataFrameIPTimeStamp.createOrReplaceTempView("log_session")

In [32]:
#Just to check if we have time_difference >15 mins or 900 secs
SpSession.sql("select IP,user_agent,time_diff_in_secs from log_session where time_diff_in_secs>900 order by IP, user_agent,time_stamp ").show()

+-------------+--------------------+-----------------+
|           IP|          user_agent|time_diff_in_secs|
+-------------+--------------------+-----------------+
|1.186.111.224|Mozilla/5.0 (Wind...|             1597|
|   1.186.37.2|Mozilla/4.0 (comp...|             6599|
|  1.186.37.25|Mozilla/5.0 (Wind...|            21898|
|   1.186.37.5|Mozilla/5.0 (Wind...|             1138|
|  1.186.41.45|Mozilla/5.0 (comp...|              976|
|   1.186.41.9|Mozilla/5.0 (Wind...|            20110|
|  1.186.76.11|Mozilla/5.0 (Wind...|              924|
|   1.186.76.2|Mozilla/5.0 (Maci...|             5520|
|   1.186.76.2|com.apple.Safari....|             5413|
|  1.186.77.10|Mozilla/5.0 (Wind...|             1603|
|  1.186.77.11|Mozilla/5.0 (Wind...|             1053|
|  1.186.77.19|Mozilla/5.0 (Wind...|             6233|
|  1.186.77.19|Mozilla/5.0 (Wind...|            18488|
|  1.186.78.18|Mozilla/5.0 (Wind...|            33170|
|  1.186.78.18|Mozilla/5.0 (Wind...|             5549|
|   1.186.

In [ ]:
#If time difference>900 secs, new_Session is set to 1 or if time_difference is null(it becomes null if previous timestamp is null
#or in other words,if the IP and user agent is different from the previous one)

logDataFrameSession = logDataFrameIPTimeStamp.withColumn('new_session',F.when(((logDataFrameIPTimeStamp.time_diff_in_secs > 900) | (logDataFrameIPTimeStamp.time_diff_in_secs.isNull())), lit(1)).otherwise(lit(0)))
logDataFrameSession.createOrReplaceTempView("log_new_session")


In [93]:
#Column new session is summed over to determine a session id which will be unique for unique set of IP, user_Agent
logDataFrameSessionId = SpSession.sql("select *,(SUM(new_session) OVER (PARTITION BY IP,user_agent ORDER BY IP,user_agent,time_stamp)) as session_id from log_new_session")
logDataFrameSessionId.createOrReplaceTempView("log_new_session_id")


In [94]:
#Session length for sessions that just start are taken as sum of request_processing_time,response_processing_time and backend_processing_time, assuming the user waits for the response
SpSession.sql("select IP,user_agent,new_session,session_id,time_stamp,ifnull(time_diff_in_secs, (log_new_session_id.request_processing_time+log_new_session_id.backend_processing_time, assuming the user waits for the response+\
          log_new_session_id.response_processing_time)) as time_diff_in_secs ,time_stamp_previous from log_new_session_id order by IP,user_agent,time_stamp").show(40)

#Q1 i.e the sessions are represented by logDataFrameSessionId


+-------------+--------------------+-----------+----------+--------------------+-------------------+--------------------+
|           IP|          user_agent|new_session|session_id|          time_stamp|  time_diff_in_secs| time_stamp_previous|
+-------------+--------------------+-----------+----------+--------------------+-------------------+--------------------+
| 1.186.101.79|Mozilla/5.0 (Wind...|          1|         1|2015-07-22 10:45:...|0.07804100000000001|                null|
| 1.186.101.79|Mozilla/5.0 (Wind...|          0|         1|2015-07-22 10:45:...|                0.0|2015-07-22 10:45:...|
| 1.186.101.79|Mozilla/5.0 (Wind...|          0|         1|2015-07-22 10:46:...|               32.0|2015-07-22 10:45:...|
| 1.186.101.79|Mozilla/5.0 (Wind...|          0|         1|2015-07-22 10:46:...|               29.0|2015-07-22 10:46:...|
| 1.186.101.79|Mozilla/5.0 (Wind...|          0|         1|2015-07-22 10:47:...|                5.0|2015-07-22 10:46:...|
| 1.186.101.79|Mozilla/5

In [95]:
SpSession.sql("select max(session_id)  from log_new_session_id ").show()

+---------------+
|max(session_id)|
+---------------+
|             13|
+---------------+



In [96]:
SpSession.sql("select * from log_new_session_id where IP is null").count()
#So, null IPs are there

0

In [97]:
logDataFrameSessionTime = SpSession.sql("select IP,session_id,sum(time_diff_in_secs) as session_time from log_new_session_id group by IP,session_id order by IP,session_id")
logDataFrameSessionTime.createOrReplaceTempView("log_session_time_tmp")
logDataFrameSessionTime= SpSession.sql("select IP, ifnull(session_time, 0) as session_time from log_session_time_tmp")
logDataFrameSessionTime.show(5)

+-------------+------------+
|           IP|session_time|
+-------------+------------+
| 1.186.101.79|        1118|
|1.186.103.240|          11|
| 1.186.103.78|           9|
|1.186.108.213|         113|
|1.186.108.230|          19|
+-------------+------------+
only showing top 5 rows



In [98]:
###################################Q2###############################################
#find average session time
logDataFrameSessionTime.createOrReplaceTempView("log_session_time")
SpSession.sql("select Avg(session_time) as avg_session_time from log_session_time").show()
#average session time is 1071.244227237604

+-----------------+
| avg_session_time|
+-----------------+
|1071.244227237604|
+-----------------+



In [99]:
###################################Q3###############################################
SpSession.sql("select distinct IP,session_id, request from log_new_session_id").take(1)
logUniqueVisits = SpSession.sql("select IP,session_id,count(*) as unique_url_count from ( select distinct IP,session_id, url from log_new_session_id )group by IP,session_id order by IP,session_id")
logUniqueVisits.show()

+-------------+----------+----------------+
|           IP|session_id|unique_url_count|
+-------------+----------+----------------+
| 1.186.101.79|         1|              12|
|1.186.103.240|         1|               5|
| 1.186.103.78|         1|               4|
|1.186.108.213|         1|               2|
|1.186.108.230|         1|               5|
|1.186.108.242|         1|               3|
| 1.186.108.28|         1|              80|
| 1.186.108.29|         1|               3|
| 1.186.108.79|         1|               3|
|1.186.111.224|         1|               9|
|1.186.111.224|         2|              12|
| 1.186.112.12|         1|               3|
| 1.186.112.52|         1|              16|
|1.186.114.135|         1|               2|
| 1.186.114.45|         1|               6|
|1.186.117.150|         1|               1|
| 1.186.128.57|         1|               8|
|1.186.129.213|         1|               6|
|1.186.135.123|         1|              11|
|1.186.135.254|         1|      

In [100]:
###################################Q4###############################################
SpSession.sql("select IP,session_time from log_session_time order by session_time desc").show()

+---------------+------------+
|             IP|session_time|
+---------------+------------+
|  116.50.59.180|      160503|
|  116.50.78.195|      152555|
|    202.3.66.50|      147032|
|  103.27.171.54|      132735|
|  103.16.70.143|      131817|
|   74.125.63.33|      124715|
|   125.19.44.66|      119790|
|  14.139.208.84|      118055|
|  123.63.241.40|      115607|
| 117.239.35.226|      112995|
|   59.144.58.37|      108711|
| 120.29.232.107|      107809|
|   59.144.58.37|      105523|
|   203.91.192.4|      105513|
|103.238.159.251|      101560|
| 117.104.239.89|      101491|
|   103.12.119.2|      100630|
|  216.150.189.2|       99609|
|   14.139.160.5|       98061|
|    61.246.57.5|       97880|
+---------------+------------+
only showing top 20 rows



In [101]:
################################ML Data prep##################################

SpSession.sql("select time_stamp_without_microsecs from df_time_stamp_without_microsecs order by time_stamp_without_microsecs asc").take(10)
requestsPrepDf = SpSession.sql("select time_stamp_without_microsecs, count(*) as requests_per_sec from df_time_stamp_without_microsecs group by time_stamp_without_microsecs order by time_stamp_without_microsecs")
SpSession.sql("select min(time_stamp_without_microsecs),max(time_stamp_without_microsecs) from df_time_stamp_without_microsecs").show(5)
requestsPrepDf.createOrReplaceTempView("df_requests_prep")
SpSession.sql("select time_stamp_without_microsecs,requests_per_sec from df_requests_prep").count()
requestsPredDf = SpSession.sql("select requests_per_sec from df_requests_prep")
df_requests_pd = requestsPredDf.toPandas()

+---------------------------------+---------------------------------+
|min(time_stamp_without_microsecs)|max(time_stamp_without_microsecs)|
+---------------------------------+---------------------------------+
|              2015-07-22 02:40:06|              2015-07-22 21:10:27|
+---------------------------------+---------------------------------+



In [102]:
# acf = plot_acf(df_requests_pd)
# acf.savefig(r'''C:\Users\arora\WeblogChallenge\images\acf.png''')
# acf = plot_pacf(df_requests_pd)
# acf.savefig(r'''C:\Users\arora\WeblogChallenge\images\pacf.png''')

In [ ]:
#to check stationarity
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

diff = df_requests_pd.values.mean()
diff_d1 = difference(df_requests_pd.values,1).mean()
diff_d1
#value of d=1 gives stationarity and minimum mean

array([0.01780694])

In [ ]:
diff_d2 = difference(df_requests_pd.values,2).mean()
diff_d2

array([0.02624795])

In [ ]:
df_requests_pd =df_requests_pd['requests_per_sec']
train = df_requests_pd[2289:4209]
test = df_requests_pd[-60:]
test

4209     96
4210     85
4211     86
4212     76
4213     75
4214     91
4215     80
4216     74
4217     69
4218     96
4219     97
4220    142
4221     89
4222     49
4223     95
4224    103
4225     97
4226     80
4227     85
4228     78
4229     72
4230     80
4231     74
4232     59
4233     73
4234     72
4235     73
4236     68
4237     61
4238     99
4239     65
4240     77
4241     67
4242     78
4243     93
4244     78
4245     93
4246     80
4247     61
4248     88
4249     79
4250     81
4251     74
4252     80
4253     82
4254    101
4255     87
4256     83
4257     77
4258    166
4259    122
4260    100
4261    107
4262     73
4263     74
4264     72
4265     99
4266     69
4267     98
4268    103
Name: requests_per_sec, dtype: int64

In [ ]:
history=[x for x in train]

In [ ]:
final_aic = float('Inf')
for p in range(0,7):
              print("p",p)
              model = ARIMA(history,order=(p, 1, 0))
              model_fit = model.fit(disp=0)
              current_aic = model_fit.aic
              print(current_aic)
              if(current_aic<final_aic):
                  final_aic = current_aic
                  final_p = p
              print("final p is",final_p)    

p 0
21508.918343480243
final p is 0
p 1
21367.259960105876
final p is 1
p 2
21295.5103821626
final p is 2
p 3
21257.98213553984
final p is 3
p 4
21242.78333084158
final p is 4
p 5
21234.632892017016
final p is 5
p 6
21236.5299616252
final p is 5


In [ ]:
model = ARIMA(history,order=(5, 1, 0))
model_fit = model.fit(disp=0)
yhat = model_fit.forecast(60)
output = yhat[0]


predictions_df = pd.DataFrame(list(output))
original_df = pd.DataFrame(list(test))
timeseries_forecasts_Df = pd.concat([original_df,predictions_df],axis=1,keys=['Original','Predicted'])
timeseries_forecasts_Df.to_csv(r'''C:\Users\arora\WeblogChallenge\results\ArimaResults.csv''')
timeseries_forecasts_Df

,Original,Predicted
,0,0
0,96,76.875490
1,85,81.138747
2,86,78.308651
3,76,77.667960
4,75,77.950081
5,91,77.712258
6,80,77.958833
7,74,78.240923
8,69,78.207507


In [ ]:
model = sarimax.SARIMAX(history,order=(3, 1, 3), seasonal_order=(3, 1, 3,60),enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit(disp=0)
current_aic = model_fit.aic
yhat = model_fit.forecast(60)


In [ ]:
output = yhat


predictions_df = pd.DataFrame(list(output))
original_df = pd.DataFrame(list(test))
timeseries_forecasts_Df = pd.concat([original_df,predictions_df],axis=1,keys=['Original','Predicted'])
timeseries_forecasts_Df.to_csv(r'''C:\Users\arora\WeblogChallenge\results\SeasonalArimaResults212.csv''')
timeseries_forecasts_Df

In [ ]:
current_aic

In [ ]:
dfIPWithAvgRPAndRB = SpSession.sql("select IP, avg(request_processing_time) as avg_req_processing_time,avg(received_bytes) as avg_received_bytes from data_With_IP group by IP")
dfIPWithAvgRPAndRB.createOrReplaceTempView("df_avg_rp_rb")
dfIPWithAvgRPAndRB.show()

In [ ]:
dfIPWithUniqueUrlCount = SpSession.sql("select IP,session_id,count(*) as unique_url_count from ( select distinct IP,session_id, url from log_new_session_id )group by IP,session_id order by IP,session_id")
dfIPWithUniqueUrlCount.createOrReplaceTempView("df_unique_url")
dfIPWithAvgUniqueUrlCount = SpSession.sql("select IP, avg(unique_url_count) as avg_unique_url_count from df_unique_url group by IP")
dfIPWithAvgUniqueUrlCount.createOrReplaceTempView("df_avg_unique_url_count")

In [ ]:
dfIPWithAvgLogSessionTime = SpSession.sql("select IP, avg(session_time) as avg_session_time from log_session_time group by IP")
dfIPWithAvgLogSessionTime.createOrReplaceTempView("df_avg_session_time")

In [ ]:
dfForClusteringWithIP = SpSession.sql("select     df_avg_rp_rb.IP,avg_req_processing_time,avg_received_bytes,avg_unique_url_count,avg_session_time\
                from   df_avg_rp_rb inner join df_avg_unique_url_count ON df_avg_rp_rb.IP = df_avg_unique_url_count.IP\
                inner join df_avg_session_time ON df_avg_rp_rb.IP = df_avg_session_time.IP")

In [ ]:
dfForClustering = SpSession.sql("select avg_req_processing_time,avg_received_bytes,avg_unique_url_count,avg_session_time\
                from   df_avg_rp_rb inner join df_avg_unique_url_count ON df_avg_rp_rb.IP = df_avg_unique_url_count.IP\
                inner join df_avg_session_time ON df_avg_rp_rb.IP = df_avg_session_time.IP").cache()

In [ ]:
summStats=dfForClustering.describe().toPandas()
meanValues=summStats.iloc[1,1:5].values.tolist()
stdValues=summStats.iloc[2,1:5].values.tolist()

#place the means and std.dev values in a broadcast variable
bcMeans=sc.broadcast(meanValues)
bcStdDev=sc.broadcast(stdValues)

def normalize(inRow) :

    global bcMeans
    global bcStdDev
    
    meanArray=bcMeans.value
    stdArray=bcStdDev.value

    retArray=[]
    for i in range(4):
        retArray.append( (float(inRow[i]) - float(meanArray[i])) /\
            float(stdArray[i]) )
    return Vectors.dense(retArray)

In [ ]:
# #ran it in py file and not running again to save time
# cost = np.zeros(8)
# for t in range(3,8):   
#    print("k is",t)
#    kmeans = KMeans(k=t, seed=1)
#    model = kmeans.fit(clusteringDataDf)
#    cost[t] = model.computeCost(clusteringDataDf)
#    print("cost for k=",t,cost[t])
# #cost for
# #                    k = 3 is 237352.30733043578
# #                    k = 4 is 172033.76753159278
# #                    k = 5 is 167181.11987527306
# #                    k = 6 is 134039.79768744443
# #                    k = 7 is 88746.62403988624
# #                    k = 8 is 133838.24873825608
# #Using elbow method , k =7 is the best for clustering

In [ ]:
clusteringData = dfForClustering.rdd.map(normalize)
clusteringData.collect()

dataRows=clusteringData.map( lambda f:Row(features=f))
clusteringDataDf = SpSession.createDataFrame(dataRows)

In [ ]:
kmeans = KMeans(k=7, seed=1)
model = kmeans.fit(clusteringDataDf)
predictions = model.transform(clusteringDataDf)

In [ ]:
import pyspark.sql.functions as func
clusteringDataDf=dfForClusteringWithIP.withColumn('row_index', func.monotonically_increasing_id())
predictions=predictions.withColumn('row_index', func.monotonically_increasing_id())
clusteringDataDfPred = clusteringDataDf.join(predictions, on=["row_index"]).sort("row_index").drop("row_index").cache()
clusteringDataDfPred.show(5)

In [ ]:
clusteringDataDfPred.createOrReplaceTempView("clustering_pred_df")
SpSession.sql("select prediction as cluster, avg(avg_session_time) as avg_session_time_from_cluster from clustering_pred_df group by prediction order by prediction").show()

In [ ]:
SpSession.sql("select IP,prediction as cluster from clustering_pred_df where prediction=2").show(1)

In [ ]:
SpSession.sql("select IP,avg_session_time as avg_session_time_from_hisory from df_avg_session_time where IP ='54.169.20.106'").show()

In [ ]:
# The predicted session time for IP= 54.169.20.106 is between  6440.91492063492 and  8181.714285714285

In [ ]:
clusteringDataDfPred.createOrReplaceTempView("clustering_pred_df")
SpSession.sql("select prediction as cluster, avg(avg_unique_url_count) as avg_unique_url_count_from_cluster from clustering_pred_df group by prediction order by prediction").show()

In [ ]:
SpSession.sql("select IP,avg_unique_url_count as avg_uniqe_url_count_from_hisory from df_avg_unique_url_count where IP ='54.169.20.106'").show()

In [ ]:
#For IP = 1.39.32.249, unique url count lies between 779 and 821